## Model

In [3]:
"""GCN using DGL nn package

References:
- Semi-Supervised Classification with Graph Convolutional Networks
- Paper: https://arxiv.org/abs/1609.02907
- Code: https://github.com/tkipf/gcn
"""
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
from dgl.nn.pytorch import GraphConv


class GCN(nn.Module):
    def __init__(self,
                 g,
                 in_feats,
                 n_hidden,
                 n_classes,
                 activation,
                 dropout=0.5):
        super(GCN, self).__init__()

        self.g = g

        self.gcn_layer1 = GraphConv(in_feats, n_hidden, activation=activation)

        self.gcn_layer2 = GraphConv(n_hidden, n_classes)

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, features):
        h = features

        h = self.gcn_layer1(self.g, h)

        h = self.dropout(h)

        h = self.gcn_layer2(self.g, h)

        return h

    def freeze_features(self, freeze):
        self.emb.weight.requires_grad = not freeze

    def freeze_graph(self, freeze):
        self.gcn_layer1.weight.requires_grad = not freeze
        self.gcn_layer2.weight.requires_grad = not freeze

## Functions

In [4]:
import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [5]:
def get_masks(n,
              main_ids,
              main_labels,
              test_ratio,
              val_ratio,
              seed=1):
    """
    Randomly splits data into train/val/test using random seed
    returns masks instead of the data itself  
    """
    train_mask = np.zeros(n)
    val_mask = np.zeros(n)
    test_mask = np.zeros(n)

    x_dev, x_test, y_dev, y_test = train_test_split(main_ids,
                                                    main_labels,
                                                    stratify=main_labels,
                                                    test_size=test_ratio,
                                                    random_state=seed)

    x_train, x_val, y_train, y_val = train_test_split(x_dev,
                                                      y_dev,
                                                      stratify=y_dev,
                                                      test_size=val_ratio,
                                                      random_state=seed)

    train_mask[x_train] = 1
    val_mask[x_val] = 1
    test_mask[x_test] = 1

    return train_mask, val_mask, test_mask


In [6]:
def evaluate(model, features, labels, mask):
    """
    Evaluate model quality (F1-score)
    """
    model.eval()
    with torch.no_grad():
        logits = model(features)
        logits = logits[mask]
        labels = labels[mask].detach().cpu().numpy()
        _, predicted = torch.max(logits, dim=1)
        predicted = predicted.detach().cpu().numpy()
        f1 = f1_score(labels, predicted, average='micro')
        return f1

In [7]:
### REPLACE HERE WITH YOUR MODEL

MODEL = GCN

In [37]:
import numpy as np
import dgl
from dgl import DGLGraph

def train_gcn_lp(graph,
              features,
              labels,
              seed=1,
              n_hidden=64,
              n_epochs=200,
              lr=1e-2,
              weight_decay=5e-4,
              dropout=0.5,
              verbose=True,
              cuda=False):

    features = torch.FloatTensor(features)
    labels = torch.LongTensor(labels)
    
    mask = []
    for i in range(len(labels)):
        # nodes with labels
        if graph.nodes[i]['is_main']:
            mask.append(1)
        else:
            mask.append(0)
            
    mask = torch.BoolTensor(mask)
        

    if cuda:
        torch.cuda.set_device("cuda:0")
        features = features.cuda()
        labels = labels.cuda()
        train_mask = train_mask.cuda()
        val_mask = val_mask.cuda()
        test_mask = test_mask.cuda()

    g = DGLGraph(graph)
    g = dgl.transform.add_self_loop(g)
    n_edges = g.number_of_edges()

    degs = g.in_degrees().float()
    norm = torch.pow(degs, -0.5)
    norm[torch.isinf(norm)] = 0

    if cuda:
        norm = norm.cuda()

    g.ndata['norm'] = norm.unsqueeze(1)

    in_feats = features.shape[1]

    # + 1 for unknown class
    n_classes = len(np.unique(labels))
    
    ##########
    ##########  HERE WE USE MODEL
    ##########
    model = MODEL(g,
                in_feats=in_feats,
                n_hidden=n_hidden,
                n_classes=n_classes,
                activation=F.relu,
                dropout=dropout)
    if cuda:
        model.cuda()

    loss_fcn = torch.nn.CrossEntropyLoss()

    # use optimizer
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                           mode='min',
                                                           factor=0.9,
                                                           patience=20,
                                                           min_lr=1e-10)

    best_f1 = -100
    # initialize graph
    dur = []
    for epoch in range(n_epochs):
        model.train()
        if epoch >= 3:
            t0 = time.time()
        logits = model(features)
        loss = loss_fcn(logits[mask], labels[mask])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch >= 3:
            dur.append(time.time() - t0)

        f1 = evaluate(model, features, labels, mask)
        scheduler.step(1 - f1)
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), 'best_model.pt')

        if verbose:
            print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | F1 {:.4f} | "
                  "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur), loss.item(),
                                                f1, n_edges / np.mean(dur) / 1000))

    model.load_state_dict(torch.load('best_model.pt'))
    
    embeddings = model.gcn_layer1(model.g, features).detach().cpu().numpy()

    return embeddings

## LP Model

In [38]:
from models.base_model import BaseModel

class GCN_Model_LP(BaseModel):
    def __init__(self, graph, features, labels=None, dim=80):
        super(GCN_Model_LP, self).__init__(graph, features, dim, labels)

    def learn_embeddings(self):
        embeddings = train_gcn_lp(self.graph, self.features, self.labels)
        self.embeddings = embeddings

## Experiments

In [39]:
from datasets import Cora, CiteseerM10, Dblp

datasets = [
   ('Cora', Cora),
   # ('CiteseerM10', CiteseerM10),
   # ('DBLP', Dblp)
]


from text_transformers import SBert, LDA, W2V, Sent2Vec, Doc2Vec, BOW, TFIDF

tasks = [
    ('GCN (W2V)', lambda ds: LpTask(ds, test_ratios, W2V, GCN_Model_LP, d=100, labels=True))
]

In [40]:
seeds = [1]
test_ratios = [0.5, 0.7, 0.9, 0.95]

In [41]:
from tqdm import tqdm
from task import LpTask

res = {}

for ds_name, ds_constr in tqdm(datasets, desc='datasets'):
    ds = ds_constr()
    for task_name, task_constr in tqdm(tasks, desc='Tasks'):
        task = task_constr(ds)
        task_res = task.evaluate()
        for test_ratio in task_res:
            scores = task_res[test_ratio]
            res[f'{1 - test_ratio:.2f} - {ds_name} - {task_name}'] = scores

        print(res)











datasets:   0%|          | 0/1 [00:00<?, ?it/s]










Tasks:   0%|          | 0/1 [00:00<?, ?it/s]











test_ratios:   0%|          | 0/4 [00:00<?, ?it/s]












seeds:   0%|          | 0/1 [00:00<?, ?it/s]/Users/mikhail-makarov/opt/anaconda3/envs/graph_text/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/mikhail-makarov/opt/anaconda3/envs/graph_text/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Time(s) nan | Loss 1.9530 | F1 0.3024 | ETputs(KTEPS) nan
Epoch 00001 | Time(s) nan | Loss 1.8601 | F1 0.3021 | ETputs(KTEPS) nan
Epoch 00002 | Time(s) nan | Loss 1.8078 | F1 0.3021 | ETputs(KTEPS) nan
Epoch 00003 | Time(s) 0.0111 | Loss 1.8001 | F1 0.3021 | ETputs(KTEPS) 717.33
Epoch 00004 | Time(s) 0.0101 | Loss 1.7741 | F1 0.3021 | ETputs(KTEPS) 792.89
Epoch 00005 | Time(s) 0.0096 | Loss 1.7514 | F1 0.3021 | ETputs(KTEPS) 832.07
Epoch 00006 | Time(s) 0.0096 | Loss 1.7234 | F1 0.3039 | ETputs(KTEPS) 831.72
Epoch 00007 | Time(s) 0.0093 | Loss 1.6966 | F1 0.3091 | ETputs(KTEPS) 855.21
Epoch 00008 | Time(s) 0.0091 | Loss 1.6575 | F1 0.3216 | ETputs(KTEPS) 875.98
Epoch 00009 | Time(s) 0.0091 | Loss 1.6459 | F1 0.3268 | ETputs(KTEPS) 882.43
Epoch 00010 | Time(s) 0.0089 | Loss 1.6179 | F1 0.3320 | ETputs(KTEPS) 895.35
Epoch 00011 | Time(s) 0.0089 | Loss 1.5871 | F1 0.3545 | ETputs(KTEPS) 901.30
Epoch 00012 | Time(s) 0.0088 | Loss 1.5532 | F1 0.3859 | ETputs(KTEPS) 910.69
Epoc

Epoch 00113 | Time(s) 0.0121 | Loss 0.5418 | F1 0.8756 | ETputs(KTEPS) 662.27
Epoch 00114 | Time(s) 0.0121 | Loss 0.5375 | F1 0.8700 | ETputs(KTEPS) 662.11
Epoch 00115 | Time(s) 0.0121 | Loss 0.5446 | F1 0.8715 | ETputs(KTEPS) 662.06
Epoch 00116 | Time(s) 0.0121 | Loss 0.5382 | F1 0.8733 | ETputs(KTEPS) 661.70
Epoch 00117 | Time(s) 0.0121 | Loss 0.5252 | F1 0.8744 | ETputs(KTEPS) 660.60
Epoch 00118 | Time(s) 0.0121 | Loss 0.5325 | F1 0.8759 | ETputs(KTEPS) 660.11
Epoch 00119 | Time(s) 0.0121 | Loss 0.5338 | F1 0.8756 | ETputs(KTEPS) 658.99
Epoch 00120 | Time(s) 0.0121 | Loss 0.5253 | F1 0.8785 | ETputs(KTEPS) 658.22
Epoch 00121 | Time(s) 0.0121 | Loss 0.5203 | F1 0.8748 | ETputs(KTEPS) 658.16
Epoch 00122 | Time(s) 0.0122 | Loss 0.5196 | F1 0.8752 | ETputs(KTEPS) 654.63
Epoch 00123 | Time(s) 0.0122 | Loss 0.5161 | F1 0.8752 | ETputs(KTEPS) 654.30
Epoch 00124 | Time(s) 0.0122 | Loss 0.5284 | F1 0.8796 | ETputs(KTEPS) 654.81
Epoch 00125 | Time(s) 0.0122 | Loss 0.5162 | F1 0.8800 | ETputs(

/Users/mikhail-makarov/opt/anaconda3/envs/graph_text/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)













seeds: 100%|██████████| 1/1 [00:06<00:00,  6.47s/it]
























test_ratios:  25%|██▌       | 1/4 [00:06<00:19,  6.47s/it]












seeds:   0%|          | 0/1 [00:00<?, ?it/s]/Users/mikhail-makarov/opt/anaconda3/envs/graph_text/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/mikhail-makarov/opt/anaconda3/envs/graph_text/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Time(s) nan | Loss 1.9418 | F1 0.3021 | ETputs(KTEPS) nan
Epoch 00001 | Time(s) nan | Loss 1.8377 | F1 0.3021 | ETputs(KTEPS) nan
Epoch 00002 | Time(s) nan | Loss 1.8096 | F1 0.3021 | ETputs(KTEPS) nan
Epoch 00003 | Time(s) 0.0176 | Loss 1.7858 | F1 0.3021 | ETputs(KTEPS) 334.72
Epoch 00004 | Time(s) 0.0163 | Loss 1.7617 | F1 0.3131 | ETputs(KTEPS) 360.51
Epoch 00005 | Time(s) 0.0179 | Loss 1.7391 | F1 0.3383 | ETputs(KTEPS) 328.66
Epoch 00006 | Time(s) 0.0182 | Loss 1.7193 | F1 0.3261 | ETputs(KTEPS) 323.68
Epoch 00007 | Time(s) 0.0187 | Loss 1.6774 | F1 0.3146 | ETputs(KTEPS) 314.21
Epoch 00008 | Time(s) 0.0203 | Loss 1.6483 | F1 0.3176 | ETputs(KTEPS) 288.94
Epoch 00009 | Time(s) 0.0209 | Loss 1.6226 | F1 0.3301 | ETputs(KTEPS) 280.91
Epoch 00010 | Time(s) 0.0212 | Loss 1.5883 | F1 0.3578 | ETputs(KTEPS) 276.86
Epoch 00011 | Time(s) 0.0213 | Loss 1.5694 | F1 0.4184 | ETputs(KTEPS) 275.98
Epoch 00012 | Time(s) 0.0216 | Loss 1.5252 | F1 0.4985 | ETputs(KTEPS) 272.40
Epoc

...

Epoch 00110 | Time(s) 0.0086 | Loss 0.6679 | F1 0.8375 | ETputs(KTEPS) 376.59
Epoch 00111 | Time(s) 0.0086 | Loss 0.6756 | F1 0.8401 | ETputs(KTEPS) 376.80
Epoch 00112 | Time(s) 0.0086 | Loss 0.6684 | F1 0.8397 | ETputs(KTEPS) 376.97
Epoch 00113 | Time(s) 0.0086 | Loss 0.6752 | F1 0.8379 | ETputs(KTEPS) 377.29
Epoch 00114 | Time(s) 0.0086 | Loss 0.6588 | F1 0.8416 | ETputs(KTEPS) 377.81
Epoch 00115 | Time(s) 0.0086 | Loss 0.6589 | F1 0.8379 | ETputs(KTEPS) 378.26
Epoch 00116 | Time(s) 0.0085 | Loss 0.6480 | F1 0.8371 | ETputs(KTEPS) 378.65
Epoch 00117 | Time(s) 0.0085 | Loss 0.6665 | F1 0.8412 | ETputs(KTEPS) 379.16
Epoch 00118 | Time(s) 0.0085 | Loss 0.6657 | F1 0.8431 | ETputs(KTEPS) 379.63
Epoch 00119 | Time(s) 0.0085 | Loss 0.6527 | F1 0.8427 | ETputs(KTEPS) 380.28
Epoch 00120 | Time(s) 0.0085 | Loss 0.6552 | F1 0.8431 | ETputs(KTEPS) 380.80
Epoch 00121 | Time(s) 0.0085 | Loss 0.6559 | F1 0.8464 | ETputs(KTEPS) 381.39
Epoch 00122 | Time(s) 0.0085 | Loss 0.6412 | F1 0.8434 | ETputs(

/Users/mikhail-makarov/opt/anaconda3/envs/graph_text/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)













seeds: 100%|██████████| 1/1 [00:04<00:00,  4.38s/it]
























test_ratios: 100%|██████████| 4/4 [00:25<00:00,  6.07s/it]













Epoch 00198 | Time(s) 0.0084 | Loss 0.5639 | F1 0.8815 | ETputs(KTEPS) 387.20
Epoch 00199 | Time(s) 0.0084 | Loss 0.5533 | F1 0.8840 | ETputs(KTEPS) 387.20













Tasks: 100%|██████████| 1/1 [03:01<00:00, 181.59s/it]




















datasets: 100%|██████████| 1/1 [03:01<00:00, 181.69s/it]











{'0.5 - Cora - GCN (W2V)': [0.7327520849128127], '0.30000000000000004 - Cora - GCN (W2V)': [0.7224478743568914], '0.09999999999999998 - Cora - GCN (W2V)': [0.6864603074331438], '0.050000000000000044 - Cora - GCN (W2V)': [0.6627767803710353]}


In [43]:
for name, scores in res.items():
    print(name, scores, np.mean(scores), np.std(scores))

0.5 - Cora - GCN (W2V) [0.7327520849128127] 0.7327520849128127 0.0
0.30000000000000004 - Cora - GCN (W2V) [0.7224478743568914] 0.7224478743568914 0.0
0.09999999999999998 - Cora - GCN (W2V) [0.6864603074331438] 0.6864603074331438 0.0
0.050000000000000044 - Cora - GCN (W2V) [0.6627767803710353] 0.6627767803710353 0.0
